In [ ]:
# ================================
# 1) Import & Load
# ================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from pathlib import Path

df = pd.read_csv(r"C:\Users\beras\Desktop\Fraud_analyst_project\bs140513_032310.csv")
print("Loaded file:", df)


In [ ]:
# ================================
# 2) Quick examination
# ================================
print("Dataset shape (rows, columns):", df.shape)
print(df.head(10))

missing_values = df.isnull().sum()
print("\nMissing values per column:\n", missing_values)

duplicate_rows = int(df.duplicated().sum())
print(f"\nNumber of duplicate rows: {duplicate_rows}")

print("\nData types:\n", df.dtypes)

print("\nNumber of unique values per column:\n", df.nunique(dropna=False).sort_values())

In [ ]:
# ============================================
# Block 3 — Minimal cleaning + gender normalization
# ============================================

# drop ZIP columns at the start of the block (both are constant)
df = df.drop(columns=['zipcodeOri', 'zipMerchant'])
print("Dropped constant ZIP columns: zipcodeOri, zipMerchant")

# 1) strip quotes/whitespace in relevant text columns
text_cols = ['customer','age','gender','merchant','category']
for c in text_cols:
    df[c] = (
        df[c].astype(str)
             .str.strip()
             .str.strip("'")
             .str.strip('"')
    )

# 2) normalize categories
df['gender']   = df['gender'].str.lower().replace({'e':'unknown', 'u':'unknown'})
df.loc[~df['gender'].isin(['m','f','unknown']), 'gender'] = 'unknown'
df['category'] = df['category'].str.lower()

# 3) convert 'age' -> numeric (nullable Int64) — may create NaN/<NA> for non-numeric
df['age'] = pd.to_numeric(df['age'], errors='coerce').astype('Int64')

# 4) derive temporal features from 'step'
df['day']  = (df['step'] // 24).astype(int)
df['hour'] = (df['step'] % 24).astype(int)

# --- sanity check
print("Shape:", df.shape)
print("Missing values per column:")
print(df.isna().sum().sort_values(ascending=False))

In [ ]:
# ============================================
# Block 3B — Fill NaNs discovered + sentinel
# ============================================

# create a flag (1 if age was missing)
df['age_unknown'] = df['age'].isna().astype('int64')

# fill 'age' with sentinel -1 (keep Int64)
df['age'] = df['age'].fillna(-1).astype('Int64')

# quick sanity check after fill
print("\nNaN in 'age' AFTER fill:", int(df['age'].isna().sum()))

In [ ]:
# ================================
# 4) EDA
# ================================
def pct(x):
    return (x * 100).round(3)

# Class balance
class_balance = pd.concat(
    [df['fraud'].value_counts().rename('count'),
     pct(df['fraud'].value_counts(normalize=True)).rename('percent')],
    axis=1
).rename_axis('fraud').reset_index()
print("\nClass balance:\n", class_balance)

plt.figure(figsize=(4,3))
sns.barplot(data=class_balance, x='fraud', y='count')
plt.title('Class balance')
plt.tight_layout()
plt.show()

# ---- global plotting params (single source of truth) ----
CATEGORICAL_FIGSIZE = (12, 5)
NUMERIC_FIGSIZE     = (6, 4)
TOP_N               = 20

# columns to plot
categorical_columns = ['category', 'merchant', 'gender', 'day', 'hour', 'step']
numeric_columns     = ['amount', 'age']

# convert day/hour to string for categorical plotting consistency
df['day']  = df['day'].astype(str)
df['hour'] = df['hour'].astype(str)


# ---------- plotting: percent within category (0..100), safe to run always ----------
def plot_categorical_distribution_percent(df, column):
    """
    For each value in `column`, compute percent of fraud vs non-fraud relative to that category's total,
    then plot percent bars (0-100). Uses TOP_N top values by count for readability.
    """
    order = df[column].astype(str).value_counts().index[:TOP_N]
    sub = df[df[column].astype(str).isin(order)].copy()
    sub[column] = sub[column].astype(str)

    # pivot: counts per (category value, fraud)
    pivot = sub.groupby([column, 'fraud']).size().unstack(fill_value=0)

    # ensure both columns 0 and 1 exist (robustness)
    for c in [0, 1]:
        if c not in pivot.columns:
            pivot[c] = 0

    # percent within category (rows sum to 100)
    pivot_percent = pivot.div(pivot.sum(axis=1), axis=0).multiply(100)
    pivot_percent = pivot_percent.reset_index().melt(id_vars=column, value_vars=[0,1],
                                                     var_name='fraud', value_name='percent')
    if column == 'merchant' or column == 'category':
        XTICKS_ROTATION = 90
    else:
        XTICKS_ROTATION = 0

    plt.figure(figsize=CATEGORICAL_FIGSIZE)
    sns.barplot(data=pivot_percent, x=column, y='percent', hue='fraud', order=order)
    plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=100))
    handles, lbls = plt.gca().get_legend_handles_labels()
    plt.legend(handles=handles, labels=[f"fraud = {l}" for l in lbls], title='')
    plt.title(f"{column} — percent by fraud within each category (top {len(order)})")
    plt.xlabel(column)
    plt.ylabel("Percent (within category)")
    plt.xticks(rotation=XTICKS_ROTATION, ha='right')
    plt.ylim(0, 100)
    plt.tight_layout()
    plt.show()


def plot_numeric_by_fraud(df, column):
    """
    Simple numeric boxplot grouped by fraud.
    The caller should decide which numeric columns to pass (e.g., 'amount_for_plot' for log10(amount)).
    """
    plt.figure(figsize=NUMERIC_FIGSIZE)
    sns.boxplot(data=df, x='fraud', y=column)
    plt.title(f"{column} by fraud label")
    plt.xlabel("fraud")
    plt.ylabel(column)
    plt.tight_layout()
    plt.show()


def summary_categorical_table(df, column):
    """
    Summary table for a categorical column (only categories with n_fraud > 0).
    Returns columns: category_value, n_total, n_fraud, fraud_rate_pct, share_of_all_frauds_pct.
    """
    total_frauds = df['fraud'].sum()

    summary_df = (
        df.groupby(column)['fraud']
          .agg(n_total='count', n_fraud='sum')
          .reset_index()
    )

    # keep only categories that actually contain frauds
    summary_df = summary_df[summary_df['n_fraud'] > 0]

    summary_df['fraud_rate_pct'] = (summary_df['n_fraud'] / summary_df['n_total'] * 100).round(3)
    summary_df['share_of_all_frauds_pct'] = (
        (summary_df['n_fraud'] / total_frauds * 100)
        .replace([np.inf, np.nan], 0)
        .round(3)
    )

    summary_df = summary_df.sort_values('fraud_rate_pct', ascending=False).rename(columns={column: 'category_value'}).reset_index(drop=True)
    return summary_df


def summary_numeric_binned_table(df, column):
    """
    Bin numeric column into `bins` and return summary table (only bins with n_fraud > 0).
    Returns columns: value_bin, n_total, n_fraud, fraud_rate_pct, share_of_all_frauds_pct.
    """
    value_bin = pd.qcut(df[column], q=4)
    total_frauds = df['fraud'].sum()

    summary_df = (
        df.groupby(value_bin)['fraud']
          .agg(n_total='count', n_fraud='sum')
          .reset_index()
          .rename(columns={'index': 'value_bin'})
    )

    # keep only bins that contain >=1 fraud
    summary_df = summary_df[summary_df['n_fraud'] > 0]

    summary_df['fraud_rate_pct'] = (summary_df['n_fraud'] / summary_df['n_total'] * 100).round(3)
    summary_df['share_of_all_frauds_pct'] = (
        (summary_df['n_fraud'] / total_frauds * 100)
        .replace([np.inf, np.nan], 0)
        .round(3)
    )

    summary_df = summary_df.sort_values('fraud_rate_pct', ascending=False).reset_index(drop=True)
    return summary_df


# Run categorical plots
print("\nCategorical percent distributions by fraud (within-category %):")
for col in categorical_columns:
    plot_categorical_distribution_percent(df, col)

# Run numeric plots
print("\nNumeric distributions by fraud:")
for col in numeric_columns:
    plot_numeric_by_fraud(df, col)

# --- Run categorical summaries for each categorical column ---
print("\n=== CATEGORICAL SUMMARIES (top rows) ===")
for col in categorical_columns:
    print(f"\n--- Summary for: {col} ---")
    cat_summary = summary_categorical_table(df, col)   # returns DataFrame (only categories with n_fraud > 0)
    print(cat_summary.head(20))                      # show top-20 rows

# --- Run numeric binned summaries for each numeric column ---
print("\n=== NUMERIC BINNED SUMMARIES (top rows) ===")
for col in numeric_columns:
    print(f"\n--- Binned summary for: {col} ---")
    num_summary = summary_numeric_binned_table(df, col)
    print(num_summary.head(20))

In [ ]:
# ================================
# 5) Feature “alerts”
# ================================
label_column = 'fraud'
global_fraud_rate = df[label_column].mean()
print(f"\nGlobal fraud rate: {global_fraud_rate*100:.3f}%")


category_stats = df.groupby('category')[label_column].agg(count='count', frauds='sum')
category_stats['rate'] = category_stats['frauds']/category_stats['count']
category_stats['lift'] = category_stats['rate']/global_fraud_rate
print(category_stats['lift'])


merchant_stats = df.groupby('merchant')[label_column].agg(count='count', frauds='sum')
merchant_stats['rate'] = merchant_stats['frauds']/merchant_stats['count']
merchant_stats['lift'] = merchant_stats['rate']/global_fraud_rate
print(merchant_stats['lift'])


# risky sets by lift + minimum support
min_support = 200
min_lift = 2.0

high_lift_categories = category_stats.query("count >= @min_support and lift >= @min_lift").index.tolist()

high_lift_merchants = merchant_stats.query("count >= @min_support and lift >= @min_lift").index.tolist()

print(f"Risky sets — categories: {len(high_lift_categories)} | merchants: {len(high_lift_merchants)}")

# amount thresholds
amount_q99_global = df['amount'].quantile(0.99)
category_amount_q95    = df.groupby('category')['amount'].quantile(0.95)
category_amount_q90    = df.groupby('category')['amount'].quantile(0.90)

# chronological order + behavioral features (burst, first-time)
df = df.sort_values(['customer','step']).reset_index(drop=True)
df['ft_mer'] = df.groupby(['customer','merchant']).cumcount() == 0

cust_step = df.groupby(['customer','step']).size().rename('cust_step_n').reset_index()
cust_step = cust_step.sort_values(['customer','step']).reset_index(drop=True)
cust_step['cust_prev6_n'] = (
    cust_step.groupby('customer')['cust_step_n']
             .transform(lambda s: s.shift().rolling(6, min_periods=1).sum())
).fillna(0).astype(int)
df = df.merge(cust_step, on=['customer','step'], how='left')

# deviation from customer's median amount
tiny_eps = 1e-9
df['cust_median_amount'] = df.groupby('customer')['amount'].transform('median').fillna(0)
df['amt_over_cust_median'] = df['amount'] / (df['cust_median_amount'] + tiny_eps)
amount_deviation_threshold = max(df['amt_over_cust_median'].quantile(0.99), 5.0)

# vectorized masks
mask_amount_above_global_q99 = df['amount'] > amount_q99_global
mask_amount_above_category_q95    = df['amount'] > df['category'].map(category_amount_q95).fillna(np.inf)
mask_amount_above_category_q90    = df['amount'] > df['category'].map(category_amount_q90).fillna(np.inf)

mask_high_lift_category  = df['category'].isin(high_lift_categories)
mask_high_lift_merchant  = df['merchant'].isin(high_lift_merchants)

mask_first_time_merchant_for_customer  = df['ft_mer']
mask_amount_deviates_from_customer_median  = df['amt_over_cust_median'] > amount_deviation_threshold

# burst (6 steps) - threshold by 0.99 quantile
burst_quantile = 0.99
burst_threshold_count = int(max(2, np.nanquantile(df['cust_prev6_n'], burst_quantile)))
mask_customer_burst  = df['cust_prev6_n'] >= burst_threshold_count
print(f"[Burst] threshold: >= {burst_threshold_count} (q={burst_quantile})")

# deterministic rules
df['rule_amount_grtr_global_q99']     = mask_amount_above_global_q99
df['rule_amount_grtr_category_q95'] = mask_amount_above_category_q95
df['rule_high_lift_category']    = mask_high_lift_category
df['rule_high_lift_merchant']    = mask_high_lift_merchant
df['rule_customer_burst6']       = mask_customer_burst
df['rule_age_missing']       = (df['age_unknown'] == 1)

# AND/OR rules
df['rule_high_merchant_high_amount'] = mask_high_lift_merchant & (df['amount'] >= 50)
df['rule_high_category_and_amount'] = mask_high_lift_category & (df['amount'] >= 42.54)
df['rule_high_cat_or_mer_and_amount'] = (mask_high_lift_category | mask_high_lift_merchant) & (df['amount'] >= 55)
df['rule_high_risk'] = (mask_high_lift_category | mask_high_lift_merchant) & (mask_amount_above_category_q90 | mask_amount_above_global_q99 | mask_first_time_merchant_for_customer | mask_customer_burst | mask_amount_deviates_from_customer_median)
df['rule_high_lift_amount_behavior'] = ((mask_high_lift_category | mask_high_lift_merchant) & (mask_amount_above_category_q95 | mask_amount_above_global_q99) & (mask_first_time_merchant_for_customer | mask_customer_burst | mask_amount_deviates_from_customer_median))


In [ ]:
# ================================
# 6) Evaluate rules (precision / recall / lift) using sklearn
# ================================

# label column (set explicitly for reproducibility)
label_column = 'fraud'

y_true = df[label_column].astype(int).to_numpy()
global_fraud_rate = y_true.mean() if len(y_true) else 0.0

def eval_rule_sklearn(y_true, y_pred_series, name):
    """
    y_true: numpy array of 0/1
    y_pred_series: pandas Series boolean/int
    returns dict with rule, flagged, tp, precision, recall, lift
    """
    y_pred = y_pred_series.astype(int).to_numpy()
    flagged = int(y_pred.sum())
    # use zero_division=0 to avoid exceptions when no positive predictions
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    lift = (precision / global_fraud_rate) if global_fraud_rate > 0 else np.nan
    return {
        'rule': name,
        'flagged': flagged,
        'tp': int(tp),
        'precision': round(precision, 4),
        'recall': round(recall, 4),
        'lift': round(lift, 2)
    }

# list of rule columns to evaluate
rule_cols = [
    'rule_amount_grtr_global_q99',
    'rule_amount_grtr_category_q95',
    'rule_high_lift_category',
    'rule_high_lift_merchant',
    'rule_customer_burst6',
    'rule_age_missing',
    'rule_high_merchant_high_amount',
    'rule_high_category_and_amount',
    'rule_high_cat_or_mer_and_amount',
    'rule_high_risk',
    'rule_high_lift_amount_behavior'
]

results = []
for rc in rule_cols:
    results.append(eval_rule_sklearn(y_true, df[rc], rc))

eval_tbl = pd.DataFrame(results).sort_values(['recall', 'precision'], ascending=[False, False]).reset_index(drop=True)
print("\nRule evaluation (sorted by recall, then precision):")
print(eval_tbl)

eval_tbl['score'] = eval_tbl['recall'] * eval_tbl['precision']
top5_rules = eval_tbl.sort_values('score', ascending=False).head(5)

print("\n=== Top 5 Rules by recall*precision score ===")
print(top5_rules)